In [1]:
# importing packages we'll need
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# reading the data set
df = pd.read_csv('loan.csv', dtype = {'issue_d': str}, low_memory=False)
# https://www.kaggle.com/wendykan/lending-club-loan-data

In [ ]:
df.head()

In [7]:
list(df)

['id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'url',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'application_type',
 'annual_inc_joint',
 'dti_joint',
 'verification_status_joint',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_

In [ ]:
df.info()

In [ ]:
df.nunique()

In [ ]:
df.isnull().sum()*100/df.isnull().count()

In [ ]:
df['dti'].corr(df['int_rate'])

In [6]:
df_num = df.select_dtypes(exclude=['object'])
df_num.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,url,dti,...,deferral_term,hardship_amount,hardship_length,hardship_dpd,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,2500,2500,2500.0,13.56,84.92,55000.0,NaN,18.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,30000,30000,30000.0,18.94,777.23,90000.0,NaN,26.52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,5000,5000,5000.0,17.97,180.69,59280.0,NaN,10.51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,4000,4000,4000.0,18.94,146.51,92000.0,NaN,16.74,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,30000,30000,30000.0,16.14,731.78,57250.0,NaN,26.35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
col_names = list(df_num)

In [11]:
for i in col_names:
    print("{} and interest rate correlation is {}".format(i, df_num[i].corr(df_num['int_rate'])))

id and interest rate correlation is nan
member_id and interest rate correlation is nan
loan_amnt and interest rate correlation is 0.06697226362378149
funded_amnt and interest rate correlation is 0.06697357995593946
funded_amnt_inv and interest rate correlation is 0.06694671610251973
int_rate and interest rate correlation is 0.9999999999999998
installment and interest rate correlation is 0.10159058867491691
annual_inc and interest rate correlation is -0.07336469442038507
url and interest rate correlation is nan
dti and interest rate correlation is 0.13085444536568852
delinq_2yrs and interest rate correlation is 0.062226268767203115
inq_last_6mths and interest rate correlation is 0.16274004714640447
mths_since_last_delinq and interest rate correlation is -0.051079784413263
mths_since_last_record and interest rate correlation is -0.018507672738113087
open_acc and interest rate correlation is -0.012564465436055211
pub_rec and interest rate correlation is 0.051749037607456215
revol_bal and 

In [14]:
corr_list = []
for i in col_names:
    corr_list.append(df_num[i].corr(df_num['int_rate']))

In [15]:
corr_list

[nan,
 nan,
 0.06697226362378149,
 0.06697357995593946,
 0.06694671610251973,
 0.9999999999999998,
 0.10159058867491691,
 -0.07336469442038507,
 nan,
 0.13085444536568852,
 0.062226268767203115,
 0.16274004714640447,
 -0.051079784413263,
 -0.018507672738113087,
 -0.012564465436055211,
 0.051749037607456215,
 -0.03632037315922344,
 0.24721626880370057,
 -0.045075764758149776,
 0.03750869061825761,
 0.03753501130796804,
 0.045871802246142075,
 0.045835297437169194,
 -0.061247265895484536,
 0.37301948801313434,
 0.06071676375108787,
 0.12325733918878794,
 0.12140202120415987,
 0.056979123360535774,
 0.020831733940865783,
 -0.03995534939376981,
 nan,
 -0.12014916443561832,
 0.2262957387843996,
 0.009393894892056021,
 0.005355187912079909,
 -0.08175852410202644,
 0.13614438284940675,
 0.04071508214085312,
 0.16258681050002674,
 0.15490519773738004,
 -0.09549040817731055,
 0.04141974234424632,
 0.14370710489644667,
 0.11334552003092069,
 0.11801192885989092,
 -0.06633084071401478,
 0.2942122

In [16]:
new_df = pd.DataFrame()

In [17]:
new_df['term'] = col_names
new_df['corr'] = corr_list
new_df.head()

,term,corr
0,id,NaN
1,member_id,NaN
2,loan_amnt,0.066972
3,funded_amnt,0.066974
4,funded_amnt_inv,0.066947


In [19]:
new_df.sort_values(by=['corr'], ascending=False)

,term,corr
5,int_rate,1.000000
100,hardship_amount,0.636597
103,orig_projected_additional_accrued_interest,0.627249
24,total_rec_int,0.373019
106,settlement_amount,0.337066
104,hardship_payoff_balance_amount,0.294277
47,all_util,0.294212
55,bc_util,0.265513
93,sec_app_revol_util,0.261570
82,percent_bc_gt_75,0.260068


In [22]:
df_obj = df.select_dtypes(include=['object'])
list(df_obj)

['term',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'earliest_cr_line',
 'initial_list_status',
 'last_pymnt_d',
 'next_pymnt_d',
 'last_credit_pull_d',
 'application_type',
 'verification_status_joint',
 'sec_app_earliest_cr_line',
 'hardship_flag',
 'hardship_type',
 'hardship_reason',
 'hardship_status',
 'hardship_start_date',
 'hardship_end_date',
 'payment_plan_start_date',
 'hardship_loan_status',
 'disbursement_method',
 'debt_settlement_flag',
 'debt_settlement_flag_date',
 'settlement_status',
 'settlement_date']

In [20]:
feat_list = ['all_util', 'percent_bc_gt_75', 'dti_joint', 'num_tl_op_past_12m', 'inq_last_6mths', 'open_il_12m', 'bc_open_to_buy', 'total_bc_limit', 'mo_sin_old_rev_tl_op', 'sec_app_mort_acc', 'annual_inc_joint', 'pub_rec_bankruptcies', 'term', 'sub_grade', ]